In [1]:
import math
import pandas as pd

## Combine and prepare price and sentiment dataframes for analysis

Here we join the price and sentiment dataframes on the date, and calculate both
the day-to-day change in closing price as well as the "Tweet Score", defined
here as the difference between the squares of the positive tweet and negative
tweet volumes.

Additionally, the "Lagged Tweet Score" is created to compare how tweets on one
day correlate with the change in closing price the next day.

In [2]:
def prep_scores(price_df, sentiment_df):
    s_df = sentiment_df.copy()
    s_df['Date'] = pd.to_datetime(s_df['Date'])
    s_df.reset_index(inplace=True)
    s_df = s_df.resample('D', on='Date').sum()

    p_df = price_df.copy()
    p_df['date'] = pd.to_datetime(p_df['date'])
    p_df.set_index('date', inplace=True)
    p_df = p_df.tz_localize(tz=None)

    combined_df = s_df.join(other=p_df)
    combined_df.dropna(inplace=True)

    combined_df['Tweet Score'] = combined_df['Positive Tweets']**2 - combined_df['Negative Tweets']**2
    combined_df['Tweet Score'] = combined_df['Tweet Score'].astype(float)
    combined_df['Tweet Score'] = combined_df['Tweet Score'].apply(
        lambda score: math.sqrt(score) if score >= 0 else -math.sqrt(-score)
    )
    combined_df['Close Diff'] = combined_df['close'] - combined_df['close'].shift(1)
    combined_df['Lagged Tweet Score'] = combined_df['Tweet Score'].shift(1)

    return combined_df

def show_correlation(df):
    display(df[['Tweet Score', 'Lagged Tweet Score', 'Close Diff']].corr())

def get_correlation_score(df):
    corr = df[['Tweet Score', 'Lagged Tweet Score', 'Close Diff']].corr()
    score = corr.iloc[0,2]
    return score

In [3]:
sentiment = pd.read_csv('./data/daily-sentiment-summary.csv')

price_all = [
    pd.read_csv('./data/amzn_stock_data.csv'),
    pd.read_csv('./data/aapl_stock_data.csv'),
    pd.concat([
        pd.read_csv('./data/goog_stock_data.csv'),
        pd.read_csv('./data/googl_stock_data.csv')
    ]),
    pd.read_csv('./data/msft_stock_data.csv'),
    pd.read_csv('./data/tsla_stock_data.csv')
]

sentiment_all = [
    sentiment.loc[sentiment['Ticker'] == 'AMZN'].copy(),
    sentiment.loc[sentiment['Ticker'] == 'APPL'].copy(),
    sentiment.loc[sentiment['Ticker'] == 'GOOG'].copy(),
    sentiment.loc[sentiment['Ticker'] == 'MSFT'].copy(),
    sentiment.loc[sentiment['Ticker'] == 'TSLA'].copy()
]

corr_scores = []

for i in range(0,5):
    combined_df = prep_scores(price_all[i], sentiment_all[i])
    show_correlation(combined_df)
    corr_scores.append(get_correlation_score(combined_df))

corr_df = pd.DataFrame({
    'Company': ['Amazon', 'Apple', 'Google', 'Microsoft', 'Tesla'],
    'Correlation Score': corr_scores
})
display(corr_df)

,Tweet Score,Lagged Tweet Score,Close Diff
Tweet Score,1.000000,0.398183,0.114406
Lagged Tweet Score,0.398183,1.000000,0.000941
Close Diff,0.114406,0.000941,1.000000


,Tweet Score,Lagged Tweet Score,Close Diff
Tweet Score,1.000000,0.557871,0.08465
Lagged Tweet Score,0.557871,1.000000,0.04943
Close Diff,0.084650,0.049430,1.00000


,Tweet Score,Lagged Tweet Score,Close Diff
Tweet Score,1.000000,0.768555,0.054653
Lagged Tweet Score,0.768555,1.000000,0.001943
Close Diff,0.054653,0.001943,1.000000


,Tweet Score,Lagged Tweet Score,Close Diff
Tweet Score,1.000000,0.472712,0.124491
Lagged Tweet Score,0.472712,1.000000,0.075104
Close Diff,0.124491,0.075104,1.000000


,Tweet Score,Lagged Tweet Score,Close Diff
Tweet Score,1.000000,0.707353,0.101101
Lagged Tweet Score,0.707353,1.000000,0.012738
Close Diff,0.101101,0.012738,1.000000


,Company,Correlation Score
0,Amazon,0.114406
1,Apple,0.084650
2,Google,0.054653
3,Microsoft,0.124491
4,Tesla,0.101101
